In [1]:
# !pip install transformers
# !pip install accelerate

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
# #양자화에 필요한 패키지 설치
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install git+https://github.com/huggingface/peft


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from torch.optim.lr_scheduler import CosineAnnealingLR

from tqdm import tqdm
from torch import nn
from transformers import Trainer
import torch.nn.functional as F
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

In [5]:
CFG = {
    'LR' : 5e-6, # Learning Rate
    'EPOCHS' : 50, # 학습 Epoch
    'BATCH_SIZE' : 1,
    'AUG_RATIO' : 0.15,
    'AUG_PROB' : 0.5,
}

In [6]:
data_df = pd.read_csv('./data/train.csv')
data_df = data_df.drop(['id','category'], axis=1)


In [7]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', eos_token='</s>')
# tokenizer = PreTrainedTokenizerFast.from_pretrained('./skt_aug2_2_29_epoch/', max_length=10000, eos_token='</s>',  truncation=True)
# model = GPT2LMHeadModel.from_pretrained("./skt_aug2_2_29_epoch/")

# tokenizer = AutoTokenizer.from_pretrained("Edentns/DataVortexS-10.7B-dpo-v1.0", eos_token='</s>', max_length=5000, truncation=True)
# model = AutoModelForCausalLM.from_pretrained("Edentns/DataVortexS-10.7B-dpo-v1.0")
model = AutoModelForCausalLM.from_pretrained("./kko_aug2_5_epoch/")
tokenizer = AutoTokenizer.from_pretrained("./kko_aug2_5_epoch/", eos_token='</s>',max_length=10000)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# loaded_model = PeftModel.from_pretrained(
#     foundation_model,    # 프롬프트 튜닝에 사용될 기본 모델
#     peft_model_path,     # Peft 모델이 저장된 위치
#     is_trainable = False # 불러온 모델은 훈련될 필요 없음
# )
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit, PeftModel, PeftConfig

model = PeftModel.from_pretrained(
    model,    # 프롬프트 튜닝에 사용될 기본 모델
    "./kko_lora_4_2_epoch/",     # Peft 모델이 저장된 위치
    is_trainable = True 
)




In [9]:
# from transformers import AutoModelForSeq2SeqLM+ 
# from peft import PeftModel, PeftConfig 
# peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"+ config = PeftConfig.from_pretrained(peft_model_id) 
# model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)+ model = PeftModel.from_pretrained(model, peft_model_id)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# model = model.to(device)  
# model.eval() 
# inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label :", return_tensors="pt")   
# with torch.no_grad():     
#     outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=10)  
#     print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])  'complaint'


In [10]:
# from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_int8_training
# lora_config = LoraConfig(r=8,
#                          lora_alpha=32,
#                          target_modules=['q_proj','v_proj'],
#                          lora_dropout=0.1,
#                          # inference_mode=False,
#                          bias="none",
#                          task_type=TaskType.CAUSAL_LM)

# model = prepare_model_for_int8_training(model)
# model = get_peft_model(model, lora_config)


In [11]:
model.print_trainable_parameters()
# model.to(device)

trainable params: 3,670,016 || all params: 6,170,172,416 || trainable%: 0.059479958622926106


In [12]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTJForCausalLM(
      (transformer): GPTJModel(
        (wte): Embedding(64512, 4096)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-27): 28 x GPTJBlock(
            (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attn): GPTJAttention(
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
               

In [13]:
import random


class HansolDataset(Dataset):
    def __init__(self, data_df, tokenizer):
        self.tokenizer = tokenizer
        self.data_df = None
        self.sentence_list = []
        self._augment_question(data_df)
        self._generate_data()
        
    def __getitem__(self, index):
        return self.sentence_list[index]
    
    def __len__(self):
        return len(self.sentence_list)

    def _augment_question(self, data_df):
        for _, row in data_df.iterrows():
            new_row = {}
            random_number = random.choice(range(0, len(data_df)))
            new_row['질문_1'] = row['질문_1'] + ' ' + data_df['질문_1'][random_number]
            new_row['질문_2'] = row['질문_2'] + ' ' + data_df['질문_2'][random_number]
            new_row['답변_1'] = row['답변_1'] + ' ' + data_df['답변_1'][random_number]
            new_row['답변_2'] = row['답변_2'] + ' ' + data_df['답변_2'][random_number]
            new_row['답변_3'] = row['답변_3'] + ' ' + data_df['답변_3'][random_number]
            new_row['답변_4'] = row['답변_4'] + ' ' + data_df['답변_4'][random_number]
            new_row['답변_5'] = row['답변_5'] + ' ' + data_df['답변_5'][random_number]
            df = pd.DataFrame(new_row,  index = [0])
            data_df = pd.concat([data_df, df], ignore_index=True)
        self.data_df = data_df.copy(deep=True)
        
    def _generate_data(self):
        for _, row in self.data_df.iterrows():
            for q_col in ['질문_1', '질문_2']:
                for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
                    # 질문과 답변 쌍을 </s> token으로 연결
                    input_text = row[q_col] + tokenizer.eos_token + row[a_col] + tokenizer.eos_token
                    input_ids = tokenizer.encode(input_text, return_tensors='pt')
                    self.sentence_list.append(input_ids)
                    


In [ ]:

model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.001)

model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    dataset = HansolDataset(data_df = data_df, tokenizer=tokenizer)
    dataloader = DataLoader(dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True) # 미니 배치 형태로 데이터 갖추기
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch[0]
        # batch.unsqueeze(0)
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")
    scheduler.step()
    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(dataloader)}")
    model.save_pretrained(f"./kko_lora_5_{epoch}_epoch")
    tokenizer.save_pretrained(f"./kko_lora_5_{epoch}_epoch")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 - Avg Loss: 0.0757: 100%|██████| 12880/12880 [1:07:53<00:00,  3.16it/s]
/usr/local/lib/python3.8/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./kakao_aug_2_epoch/ - will assume that the vocabulary was not modified.
  warnings.warn(


Epoch 1/50, Average Loss: 0.07565374240237259


Epoch 2 - Avg Loss: 0.0770:  16%|█▍       | 2040/12880 [10:31<41:26,  4.36it/s]